In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)

# Import danych

In [2]:
def read_data(path:str):
    data = pd.read_csv(path, sep="  ", skiprows=1, header=None)

    column_name = ["Class"]

    for index in range(1, len(data.columns)):
        column_name.append(f"P{index}")

    data.columns = column_name
    return data

In [3]:
train_file_path = "../data/iris_trn.txt"
test_file_path = "../data/iris_tst.txt"

train_data = read_data(train_file_path)
test_data = read_data(test_file_path)

train_data

/opt/conda/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,Class,P1,P2,P3,P4
0,1,5.1,3.5,1.4,0.2
1,1,4.9,3.0,1.4,0.2
2,1,4.7,3.2,1.3,0.2
3,1,4.6,3.1,1.5,0.2
4,1,5.0,3.6,1.4,0.2
5,1,5.4,3.9,1.7,0.4
6,1,4.6,3.4,1.4,0.3
7,1,5.0,3.4,1.5,0.2
8,1,4.4,2.9,1.4,0.2
9,1,4.9,3.1,1.5,0.1


In [19]:
test_data

,Class,P1,P2,P3,P4
0,1,5.0,3.0,1.6,0.2
1,1,5.0,3.4,1.6,0.4
2,1,5.2,3.5,1.5,0.2
3,1,5.2,3.4,1.4,0.2
4,1,4.7,3.2,1.6,0.2
5,1,4.8,3.1,1.6,0.2
6,1,5.4,3.4,1.5,0.4
7,1,5.2,4.1,1.5,0.1
8,1,5.5,4.2,1.4,0.2
9,1,4.9,3.1,1.5,0.2


# Zadanie 1

In [5]:
gravity_centers = train_data.groupby(train_data.Class).mean()
gravity_centers

,P1,P2,P3,P4
Class,,,,
1,5.028,3.480,1.460,0.248
2,6.012,2.776,4.312,1.344
3,6.576,2.928,5.640,2.044


In [6]:
train_data_no_class = train_data.drop(columns=["Class"])
test_data_no_class = test_data.drop(columns=["Class"])

In [7]:
test_data

,Class,P1,P2,P3,P4
0,1,5.0,3.0,1.6,0.2
1,1,5.0,3.4,1.6,0.4
2,1,5.2,3.5,1.5,0.2
3,1,5.2,3.4,1.4,0.2
4,1,4.7,3.2,1.6,0.2
5,1,4.8,3.1,1.6,0.2
6,1,5.4,3.4,1.5,0.4
7,1,5.2,4.1,1.5,0.1
8,1,5.5,4.2,1.4,0.2
9,1,4.9,3.1,1.5,0.2


In [8]:
mv = train_data_no_class.mean().round(3)
mv

P1    5.872
P2    3.061
P3    3.804
P4    1.212
dtype: float64

In [9]:
sd = train_data_no_class.std().round(3)
sd

P1    0.857
P2    0.468
P3    1.815
P4    0.770
dtype: float64

In [10]:
test_data_no_class

,P1,P2,P3,P4
0,5.0,3.0,1.6,0.2
1,5.0,3.4,1.6,0.4
2,5.2,3.5,1.5,0.2
3,5.2,3.4,1.4,0.2
4,4.7,3.2,1.6,0.2
5,4.8,3.1,1.6,0.2
6,5.4,3.4,1.5,0.4
7,5.2,4.1,1.5,0.1
8,5.5,4.2,1.4,0.2
9,4.9,3.1,1.5,0.2


In [11]:
gravity_centers_std = ((gravity_centers-mv)/sd).round(3)
gravity_centers_std

,P1,P2,P3,P4
Class,,,,
1,-0.985,0.895,-1.291,-1.252
2,0.163,-0.609,0.280,0.171
3,0.821,-0.284,1.012,1.081


In [12]:
test_data_std = (test_data_no_class-mv)/sd
test_data_std

,P1,P2,P3,P4
0,-1.017503,-0.130342,-1.214325,-1.314286
1,-1.017503,0.724359,-1.214325,-1.054545
2,-0.784131,0.938034,-1.269421,-1.314286
3,-0.784131,0.724359,-1.324518,-1.314286
4,-1.367561,0.297009,-1.214325,-1.314286
5,-1.250875,0.083333,-1.214325,-1.314286
6,-0.550758,0.724359,-1.269421,-1.054545
7,-0.784131,2.220085,-1.269421,-1.444156
8,-0.434072,2.433761,-1.324518,-1.314286
9,-1.134189,0.083333,-1.269421,-1.314286


In [13]:
distances = pd.DataFrame()
for index, gravity_center_std in enumerate(gravity_centers_std.values):
    data = []
    for train_data_value in test_data_std.values:
        data.append( 
            np.sum((gravity_center_std - train_data_value) ** 2)
        )
    distances[f"D{index+1}"] = data
distances

,D1,D2,D3
0,1.062141,6.061782,14.097621
1,0.075042,6.906402,13.913958
2,0.046546,7.897152,15.012090
3,0.074470,7.455454,14.789942
4,0.513705,7.602550,15.821288
5,0.739251,6.917449,15.121517
6,0.257138,6.189966,12.663947
7,1.833589,13.910216,20.428185
8,2.676309,14.395439,20.158139
9,0.685405,6.768805,14.899976


In [14]:
distances.dtypes

D1    float64
D2    float64
D3    float64
dtype: object

In [15]:
classes = []
mins = []
for train_data_value in test_data_std.values:
    dists_to_center = []
    for gravity_center_std in gravity_centers_std.values:
        dists_to_center.append( 
            np.sum((gravity_center_std - train_data_value) ** 2)
        )
    classes.append(np.argmin(dists_to_center)+1) # argmin returns index of min element
    mins.append(dists_to_center)

test_data_std["Dist"] = mins
test_data_std["AssignClass"] = classes
test_data_std

,P1,P2,P3,P4,Dist,AssignClass
0,-1.017503,-0.130342,-1.214325,-1.314286,"[1.06214096647426, 6.061781797437175, 14.09762...",1
1,-1.017503,0.724359,-1.214325,-1.054545,"[0.07504214183963798, 6.906402364521944, 13.91...",1
2,-0.784131,0.938034,-1.269421,-1.314286,"[0.04654556406444236, 7.897151919254789, 15.01...",1
3,-0.784131,0.724359,-1.324518,-1.314286,"[0.0744698002044022, 7.455454060318107, 14.789...",1
4,-1.367561,0.297009,-1.214325,-1.314286,"[0.5137054509321101, 7.6025503230243885, 15.82...",1
5,-1.250875,0.083333,-1.214325,-1.314286,"[0.7392509262303919, 6.917449437034535, 15.121...",1
6,-0.550758,0.724359,-1.269421,-1.054545,"[0.2571380046342566, 6.189965900762595, 12.663...",1
7,-0.784131,2.220085,-1.269421,-1.444156,"[1.8335894839954727, 13.910216485206464, 20.42...",1
8,-0.434072,2.433761,-1.324518,-1.314286,"[2.6763086827499696, 14.395439329084741, 20.15...",1
9,-1.134189,0.083333,-1.269421,-1.314286,"[0.6854052873005878, 6.76880542721004, 14.8999...",1


In [16]:
np.sum(test_data_std.AssignClass==test_data.Class)

67

In [18]:
75-67

8

# Zadanie 2

In [34]:
weigth_before_standarization = 2*gravity_centers_std
column_count = len(weigth_before_standarization.columns)
weigth_before_standarization.columns = [f"w{index+1}" for index in range(column_count)]
last_w = []
for value in gravity_centers_std.values:
    last_w.append(
        -np.sum(np.power(value,2))
    )
last_w = np.array(last_w)
weigth_before_standarization

,w1,w2,w3,w4
Class,,,,
1,-1.970,1.790,-2.582,-2.504
2,0.326,-1.218,0.560,0.342
3,1.642,-0.568,2.024,2.162


In [ ]:
weigth_after_standarization